### TODO Recording:

- Go to https://mvnrepository.com/artifact/org.apache.spark/spark-sql-kafka-0-10
- Show the versions available

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkKafkaBidirectionalStreamingDemo") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.4") \
    .getOrCreate()

print("Spark version:", spark.version)

25/02/13 11:08:53 WARN Utils: Your hostname, Jananis-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.68.52 instead (on interface en0)
25/02/13 11:08:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/janani/.ivy2/cache
The jars for the packages stored in: /Users/janani/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-762c9128-12f2-447e-90a6-5755be9c85ea;1.0
	confs: [default]


:: loading settings :: url = jar:file:/opt/homebrew/Cellar/apache-spark/3.5.4/libexec/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.4 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.4 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 215ms :: artifacts dl 7ms
	:: modules in use:
	com.google.code.findbugs#jsr305;3.0.0 from central in [default]
	commons-logging#commons-logging;1.1.3 from central in [default]
	org.apache.commons#commons-pool2;2.11.1 from central in [default]
	org.apache.hadoop#hadoop-client-api;3.3.4 from central in [default]
	org.apache.hadoop#h

Spark version: 3.5.4


### Read Messages from Kafka


In [2]:
df_kafka = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "hello-kafka") \
    .load()

df_kafka_string = df_kafka.selectExpr("CAST(value AS STRING) as message")

In [3]:
def print_batch(df, epoch_id):
    if df.count() > 0:  
        df.show(truncate=False)

In [4]:
kafka_input_query = df_kafka_string.writeStream \
    .foreachBatch(print_batch) \
    .outputMode("append") \
    .option("startingOffsets", "earliest") \
    .trigger(processingTime="5 seconds") \
    .start()



25/02/13 11:12:10 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/f4/wmvdntf96vjfrq3sb2p4ncy00000gn/T/temporary-9207248e-bc5c-46ab-b6ea-78e5d89e9804. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/02/13 11:12:10 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/02/13 11:12:10 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


+-------------------------------+
|message                        |
+-------------------------------+
|Streaming into Spark from Kafka|
+-------------------------------+



+-----------------+
|message          |
+-----------------+
|Multiple messages|
|Message 2        |
+-----------------+



In [ ]:
kafka_input_query.stop()